In [ ]:
import pandas as pd
import re
import unicodedata
pd.set_option('display.max_rows', 100)

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
import os
import torch
from transformers import  AutoTokenizer

In [ ]:
data_name="Nairaland_v1"

In [ ]:

def is_link(text):
    text=str(text)
    pattern = re.compile(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+')
    return bool(pattern.match(text))

In [ ]:
def has_link(text):
    text=str(text)
    link_match = re.search(r'http[s]?:\/\/\S+', text)
    if link_match:
        return True
    return False

In [ ]:
import re

def has_email_or_mobile_number(text):
  """
  Checks if a text has a link or a mobile number (starting with 08, 07, 09 or 234 or +234)

  Args:
    text: The text to check

  Returns:
    True if the text has a link or a mobile number, False otherwise
  """
  # Check for links using regex
  text=str(text)
  #link_match = re.search(r'http[s]?:\/\/\S+', text)

  # Check for mobile numbers starting with 08, 07, 09 or 234 or +234
  mobile_number_match = re.search(r'\b(?:\+?(\d{1,3}))?[-. (]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})\b',text)#

  mobile_number_match_2=re.search(r"\b234|\b081|\b080|\b090|\b091|\b070|\+234\+wa.me", text)
  email_match=re.search(r'[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}',text)

  # Return True if there is a link match or a mobile number match
  if mobile_number_match or email_match or mobile_number_match_2:
    return True
  return False

In [ ]:
def remove_newlines(text):
    # Define the regular expression pattern
    pattern = r'(?<=[^\n])\n(?![.,?!])'

    # Replace newline characters not preceded by another newline and not followed by certain symbols with a space
    modified_text = re.sub(pattern, ' ', text)

    return modified_text

In [ ]:
import re

def remove_text_in_square_brackets(text):
    text=str(text)
    # Regular expression pattern to match text within square brackets (emojis/links/some usernames)
    pattern = r'!\[(.*?)\]\(.*?\)'

    # Remove all matches
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text.strip()

In [ ]:
def remove_links(text):
  text=str(text)
  # Define the pattern to match URLs
  url_pattern = r'https?://\S+|www\.\S+'

  # Replace URLs with an empty string
  clean_text = re.sub(url_pattern, '', text)
  return clean_text

def remove_start_usernames(text):
  """Instasnces where the texts starts with usernames for example
  'Asgard13:Again most useless tribal threads be taken down with seconds '
  """
  text=str(text)
  words_list=text.split(" ")
  if ":" in words_list[0]:
    words_list[0]="".join(words_list[0].split(":")[1:])
    return " ".join(words_list)
  return text


def convert_unicode_to_ascii(text):
    # Normalize the text to decompose any combined characters
    normalized_text = unicodedata.normalize('NFKD', text)

    # Convert the normalized text to ASCII, ignoring any characters that cannot be represented in ASCII
    ascii_text = normalized_text.encode('ascii', 'ignore').decode('ascii')

    return ascii_text

In [ ]:
def replace_links_with_placeholder(text):
    text=str(text)
    # Regular expression pattern to match URLs
    url_pattern = r'https?://\S+'
    # Replace URLs with [link]
    processed_text = re.sub(url_pattern, '[link]', text)
    return processed_text

In [ ]:
def remove_nairaland_specific_words(text):
  text=str(text).lower()

  if ("n'landers" in text) or ("lala" in text) or ("nairaland" in text) or ("nairalanders" in text ) or ("lalasticlala" in text) or ("mynd44" in text) or ("seun" in text):
    return True
  return False

In [ ]:
def remove_question_from_response(row):

  if row["content"] in row["response"]:
    row["response"]=row["response"].replace(row["content"],"")
  return row["response"]

In [ ]:
def remove_text_in_square_brackets2(text):
    # Regular expression pattern to match text within square brackets in the format "[**text**](link)"
    pattern = r'\[\*\*(.*?)\*\*\]\(.*?\)'

    # Remove all matches
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [ ]:
def not_good_response(text):
    if " OP " in text:
      return True
    text=str(text).lower()
    if ("watch the video below" in text) or ("drop ur whatsapp number" in text) or ("k070" in text) or ("k080" in text) or ("k081" in text) or ("k090" in text) or ("dick" in text) or ("pussy" in text ) or ("pvssy" in text) or ("enjoy best odds" in text) or ("kpekus" in text) or ("check my signature" in text) or ("must watch video!" in text) or (" download now." in text) or ("moniker " in text ) or ("we are not quarrelling. i only need suggestions" in text):
        return True
    return False

def not_good_question(text):
    text=str(text).lower()
    if ("call or whatsapp 0" in text) or ("dick" in text) or ("check my signature" in text) or  ("must watch video!" in text) or ("moniker " in text ) or ("read more" in text) or ("watch this" in text) or ("click here to watch" in text) or (len(text.split(" "))<5):
        return True
    return False

In [ ]:
def clean_text(text):
  text=str(text)

  # Assuming text_with_encoding is a byte string
  #text = text.decode('utf-8')
  #text=text.replace("â\x80\x99", "'")
  #text=text.replace("â\x80\x9c","“")
  try:

    text=text.replace("\r","")
    text=convert_unicode_to_ascii(text)
    if "鵪" in text:
      print(text)
    #return emojis
    text=remove_text_in_square_brackets(text)
    text=replace_links_with_placeholder(text)
    text=remove_text_in_square_brackets2(text)
    text=text.strip()
    #text=remove_links(text)
    text=remove_start_usernames(text)
    text=text.replace("> :","")
    text=text.replace(">   ","")
    text=text.replace("> ","")
    text.replace("i12i12i12","")
    text=remove_newlines(text).strip()
  except:
    print(1)
    pass
  return text

In [ ]:
def create_instruction(row):
    question,response=row["content"],row["response"]
    formatted_prompt = f"### BEGIN CONVERSATION ###\n\n## User: ##\n{question}\n\n## Assistant: ##\n{response} ### END CONVERSATION ###"
    return formatted_prompt

In [ ]:
prompt = "Your text here."
formatted_prompt = f"### BEGIN CONVERSATION ###\n\n## Speaker_0: ##\n{prompt}\n\n## Walls1337bot: ##\n"

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
model_name ="meta-llama/Meta-Llama-3-8B"#"meta-llama/Meta-Llama-3-8B-Instruct"
# "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def tokenise(text):
    return len(tokenizer.encode(text))

In [ ]:
dropped_vals=[".",".."," ","  ","...","nan","","x"]

## Question - Answer Dataset

In [ ]:
sections=["Politics","Romance","Career","Business","Education","Religion","Sports","Literature","Fashion","TV-Movies","Travel",
          "Programming","Phones","Music-Radio","Food","Family","Health","Music-Radio2000","Health2000","Family2000","Education2000","Travel2000","Sports2000","Romance2000","Politics2000","Career2000","Religion2000"]
all_df=[]
for section in sections:
  path=f"/content/drive/MyDrive/Nairaland_data/data_{section}.csv"
  df=pd.read_csv(path , lineterminator='\n',encoding = "ISO-8859-1",on_bad_lines="skip")
  df["Section"]=section
  all_df.append(df)
final_df=pd.concat(all_df)

In [ ]:
final_df["Section"]=final_df["Section"].replace({"Music-Radio2000":"Music-Radio",
                                                 "Health2000":"Health",
                                                 "Family2000":"Family",
                                                 "Education2000":"Education",
                                                 "Travel2000":"Travel",
                                                 "Sports2000":"Sports",
                                                 "Romance2000":"Romance",
                                                 "Politics2000":"Politics",
                                                 "Career2000":"Career",
                                                 "Religion2000":"Religion"})

In [ ]:
final_df.shape#(635041, 6)

(704059, 6)

In [ ]:
final_df=final_df[final_df["content"]!=final_df["response"]]

In [ ]:
#texts that occur more than once
spam_posts=final_df["content"].value_counts()[final_df["content"].value_counts()>1].index
spam_responses=final_df["response"].value_counts()[final_df["response"].value_counts()>1].index
list_spam=list(spam_posts)+list(spam_responses)

In [ ]:
final_df=final_df[~(final_df["content"].isin(list_spam))]
#=final_df[~(final_df["content"].isin(spam_responses) & final_df["post has_number_or_link"]==True )]
final_df=final_df[~(final_df["response"].isin(list_spam))]
#final_df=final_df[~(final_df["response"].isin(spam_responses) & final_df["post has_number_or_link"]==True )]

In [ ]:
final_df.shape

(472094, 6)

In [ ]:
final_df.columns

Index(['Unnamed: 0', 'title', 'content', 'response', 'likes', 'Section'], dtype='object')

In [ ]:
final_df=final_df[['title', 'content', 'response', 'likes', 'Section']]
final_df=final_df[final_df["likes"]>0]

In [ ]:
final_df=final_df[~final_df["content"].isin(dropped_vals)]
final_df=final_df[~final_df["response"].isin(dropped_vals)]

In [ ]:
instruction_copy=final_df.copy()

In [ ]:
#remove question or response that are links
final_df=final_df[ ~(final_df["content"].apply(is_link) | final_df["response"].apply(is_link))]


final_df=final_df[~final_df["response"].apply(has_email_or_mobile_number)]
final_df=final_df[~final_df["content"].apply(has_email_or_mobile_number)]
final_df=final_df[~final_df["response"].apply(has_link)]



final_df=final_df[~final_df["content"].apply(has_link)]


final_df["content"]=final_df["content"].apply(clean_text)
final_df["response"]=final_df["response"].apply(clean_text)

#remove question from response
final_df["response"]=final_df.apply(remove_question_from_response,axis=1)

final_df=final_df[~final_df["response"].apply(not_good_response)]
final_df=final_df[~final_df["content"].apply(not_good_question)]

#final_df.drop_duplicates(inplace=True)

In [ ]:
#remove_nairaland specific words
final_df=final_df[~final_df["content"].apply(remove_nairaland_specific_words)]
final_df=final_df[~final_df["response"].apply(remove_nairaland_specific_words)]


final_df=final_df[~pd.isna(final_df["content"])]
final_df=final_df[~pd.isna(final_df["response"])]

final_df=final_df[~(final_df["content"]=="nan")]
final_df=final_df[~(final_df["response"]=="nan")]

In [ ]:
final_df["instruct"]=final_df.apply(create_instruction,axis=1)

In [ ]:
final_df["length"]=final_df["instruct"].apply(tokenise)

In [ ]:
# prompt: randomly select and print a row in the overall_df dataframe, print the full content of the texts column

import pandas as pd
import numpy as np

# Select a random row from the overall_df dataframe
random_row = np.random.randint(0, len(final_df))

# Print the full content of the texts column for the selected row
print(final_df.iloc[random_row]["instruct"])

### BEGIN CONVERSATION ###

## User: ##
Titanic is a 1997 American epic romance and disaster film directed, written, co-produced and co-edited by James Cameron. A fictionalized account of the sinking of the RMS Titanic, it stars Leonardo DiCaprio and Kate Winslet as members of different social classes who fall in love aboard the ship during its ill-fated maiden voyage.Production began in 1995, when Cameron shot footage of the actual Titanic wreck. The modern scenes on the research vessel were shot on board the Akademik Mstislav Keldysh, which Cameron had used as a base when filming the wreck. Scale models, computer-generated imagery, and a reconstruction of the Titanic built at Baja Studios were used to re-create the sinking.

## Assistant: ##
D only movie that made me shed small tears as a teenager. Jack mumu for dat movie no be small. ### END CONVERSATION ###


In [ ]:
final_df["length"].mean()#69624, 6)

346.75350258084904

In [ ]:
final_df["length"].max()#

22586

In [ ]:
final_df["length"].min()#

29

In [ ]:
final_df["length"].median()#

204.5

In [ ]:
final_df.shape#69624, 6)

(75944, 7)

In [ ]:
final_df.to_csv(f"/content/drive/MyDrive/Nairaland_data/{data_name}_instructQA.csv")

In [ ]:
final_df=final_df[final_df["length"]<=512]

In [ ]:
final_df.shape#(61545, 7)

(61543, 7)

In [ ]:
final_df.to_csv(f"/content/drive/MyDrive/Nairaland_data/{data_name}_instruct_512QA.csv")

In [ ]:
random_row = np.random.randint(0, len(final_df))

final_df["instruct"].iloc[random_row]#

'### BEGIN CONVERSATION ###\n\n## User: ##\n~~~Daily Spiritual Digest~~~ ________________________________ [03-12-2018]Focus: Humanity Comes First in Everything WORD FOR TODAY Irrespective of who you are, your tribe, race, colour, country and religion; the fact still remains that humanity comes first in everything. The values of a man will not be right until he comes to consider humanity in his service towards God!!! ************************************Scriptural Reference"Love one another with brotherly affection [as members of one family], giving precedence and showing honor to one another." (Romans 12:10 AMP)~~~~~~~~~~~~~~~~~~~~~~~There was a time the pharisees accused Jesus of healing people on the Sabbath day because it was contrary to the law. But Jesus made it clear to them that humanity comes before any principles, worship, devotion. Anything you claim to do for God that is detrimental to the well being of people is not of God. The call to salvation is a call to quality and fait

In [ ]:
final_df["instruct"].value_counts()

instruct
### BEGIN CONVERSATION ###\n\n## User: ##\nHow can you be telling me how much to budget for feeding when you don't know the current price of garri ?I hate nonsense\n\n## Assistant: ##\nUna go soon stress shege comot for chatGPT body with una Nigerian problems. Chat Gbt upon receiving a request from a naijarian.... ### END CONVERSATION ###                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
final_df["Section"].value_counts()

Section
Politics       16289
Romance        15657
Religion        7846
Sports          3976
Education       3558
Travel          2917
Phones          2466
Family          2187
Career          1395
Programming     1221
Fashion          815
Business         803
TV-Movies        784
Health           611
Food             587
Music-Radio      431
Name: count, dtype: int64

In [ ]:
final_df["likes"].value_counts()

likes
1.0      20377
2.0       8934
3.0       4992
4.0       3374
5.0       2406
         ...  
826.0        1
837.0        1
675.0        1
743.0        1
389.0        1
Name: count, Length: 810, dtype: int64

## PUSH TO HUB

In [ ]:
huggingface_hub.login()

In [ ]:
file_name="Nairaland_v1_instructQA"
dataset = load_dataset("csv", data_files=f"/content/drive/MyDrive/Nairaland_data/{file_name}.csv")
dataset=dataset.remove_columns(["Unnamed: 0", 'length'])
dataset.push_to_hub(f"saheedniyi/{file_name}", private=True)

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/76 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/463 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/saheedniyi/Nairaland_v1_instructQA/commit/37a20bc77f053ccd638179b36430b02c433054de', commit_message='Upload dataset', commit_description='', oid='37a20bc77f053ccd638179b36430b02c433054de', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
file_name="Nairaland_v1_instruct_512QA"
dataset = load_dataset("csv", data_files=f"/content/drive/MyDrive/Nairaland_data/{file_name}.csv")
dataset=dataset.remove_columns(["Unnamed: 0", 'length'])
dataset.push_to_hub(f"saheedniyi/{file_name}", private=True)

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/462 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/saheedniyi/Nairaland_v1_instruct_512QA/commit/757eba32d88bbe6cfb8de71f5e927a6226f2bb3f', commit_message='Upload dataset', commit_description='', oid='757eba32d88bbe6cfb8de71f5e927a6226f2bb3f', pr_url=None, pr_revision=None, pr_num=None)